In [1]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print('length of dataset (chs): ', len(text))

length of dataset (chs):  1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
#create a mapping  from chars to ints

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
#encode the entire dataset into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
#train/test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'input: {context} | output: {target}')


input: tensor([18]) | output: 47
input: tensor([18, 47]) | output: 56
input: tensor([18, 47, 56]) | output: 57
input: tensor([18, 47, 56, 57]) | output: 58
input: tensor([18, 47, 56, 57, 58]) | output: 1
input: tensor([18, 47, 56, 57, 58,  1]) | output: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]) | output: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) | output: 58


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print(f'input: \n{xb}')
print(f'target: \n{yb}')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'input: {context.tolist()} --> target: {target}')

input: 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target: 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
input: [24] --> target: 43
input: [24, 43] --> target: 58
input: [24, 43, 58] --> target: 5
input: [24, 43, 58, 5] --> target: 57
input: [24, 43, 58, 5, 57] --> target: 1
input: [24, 43, 58, 5, 57, 1] --> target: 46
input: [24, 43, 58, 5, 57, 1, 46] --> target: 43
input: [24, 43, 58, 5, 57, 1, 46, 43] --> target: 39
input: [44] --> target: 53
input: [44, 53] --> target: 56
input: [44, 53, 56] --> target: 1
input: [44, 53, 56, 1] --> target: 58
input: [44, 53, 56, 1, 58] --> target: 46
input: [44, 53, 56, 1, 58, 46] --> target: 39
input: [44, 53, 56, 1, 58, 46, 39] --> target: 58
input: [44, 53, 56, 1, 58, 46, 39, 58] --> target: 1
input:

In [11]:
print(xb) # input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


Implementation of bigram language model

In [23]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token reads logits for next tkn from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx and targets are (B, T) tensor of ints (Batch, Time, Channel)
        #in this context time represents the sequential nature of the data (block_size). Channel is representative of the logits for each token in the embedding table(vocab_size)
        logits = self.token_embedding_table(idx) # (B,T,C) 
        
        if targets is None:
            loss = None
        else:
        #we reshape the logits and targets in order to use the cross_entropy function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self,idx, max_new_tokens):
        #idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            #focus on last time step (last element)
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax
            probs = F.softmax(logits, dim=-1) #(B,C)
            #sample from prob distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled idx to the sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx

    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
#we expect an initial logloss = -ln(1/65) =4.17  | Initial loss = 4.87

#generate from the 
idx = torch.zeros((1,1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [24]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) 

In [29]:
batch_size = 32
for steps in range(10000):

    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.465181589126587


In [31]:
idx = torch.zeros((1,1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))



Cld IVI bl; tisheer lid al'st bal l are, tour bu olll pt my.
FR tr beser fare, wer

Whoto y oo nd plerlethor myseres: omathita illt w,
Taley myocomeat nethest,
NGLAnd,
Anen!
3QUMI; frthily, my Ise IIAn,
Five.
BOUCARI ke y vese o ye foy VENRYBireswo n

ABaXqusune y my n;
'e w: t w; MELULo INen meng 
